In [1]:
import models

In [2]:
models.compare_mc_to_sim_sojourn(
    num_classes=2,
    num_servers=1,
    arrival_rates=[4, 5],
    service_rates=[12, 13],
    thetas=[[None, 2], [1, None]],
    bound=15,
    max_simulation_time=20000,
    warmup=200,
)

100%|██████████| 20000.0/20000 [00:09<00:00, 2026.70it/s]           


{'Markov Chain': [0.18902347102776995, 0.3986866457192421],
 'Simulation': [0.19387962688300975, 0.4096967517870087]}

In [3]:
models.compare_mc_to_sim_sojourn(
    num_classes=2,
    num_servers=1,
    arrival_rates=[3, 5],
    service_rates=[12, 13],
    thetas=[[None, 2], [4, None]],
    bound=15,
    max_simulation_time=20000,
    warmup=200,
)

100%|██████████| 20000.0/20000 [00:09<00:00, 2172.29it/s]           


{'Markov Chain': [0.18089660366279464, 0.27335350278687726],
 'Simulation': [0.18272704820161545, 0.27482538387242944]}

In [4]:
models.compare_mc_to_sim_sojourn(
    num_classes=3,
    num_servers=1,
    arrival_rates=[0.5, 1.3, 0.6],
    service_rates=[9, 9, 8],
    thetas=[[None, 1, 0.6], [1.1, None, 0.7], [1.2, 1, None]],
    bound=8,
    max_simulation_time=20000,
    warmup=200,
)

100%|██████████| 20000.0/20000 [00:02<00:00, 7289.59it/s]           


{'Markov Chain': [0.1204092506720846, 0.1529471254170382, 0.19299607180496842],
 'Simulation': [0.12068273808580142, 0.154405733903177, 0.19563222490353108]}

In [5]:
models.compare_mc_to_sim_sojourn(
    num_classes=2,
    num_servers=2,
    arrival_rates=[3, 4],
    service_rates=[6.5, 7.5],
    thetas=[[None, 2], [4, None]],
    bound=15,
    max_simulation_time=50000,
    warmup=200,
)

100%|██████████| 50000.0/50000 [00:16<00:00, 2950.46it/s]           


{'Markov Chain': [0.1794581385915189, 0.21367804897072376],
 'Simulation': [0.17848945316697193, 0.21354310076383318]}

In [6]:
models.compare_mc_to_sim_sojourn(
    num_classes=3,
    num_servers=2,
    arrival_rates=[1, 0.8, 0.5],
    service_rates=[3, 2.5, 2],
    thetas=[[None, 1, 0.5], [0.5, None, 0.5], [1, 0.7, None]],
    bound=8,
    max_simulation_time=20000,
    warmup=200,
)

100%|██████████| 20000.0/20000 [00:03<00:00, 6636.28it/s]           


{'Markov Chain': [0.352441377466164, 0.5031135932339846, 0.6393170884752223],
 'Simulation': [0.35245868593884844, 0.5075393946421101, 0.6364805806806387]}

# Compare sim to MC for variance

In [23]:
import numpy as np

In [24]:
def get_sojourn_var_mc(
    num_classes, num_servers, arrival_rates, service_rates, thetas, bound
):
    state_space, transmat = models.build_state_space_and_transition_matrix_sojourn_mc(
        num_classes=num_classes,
        num_servers=num_servers,
        arrival_rates=arrival_rates,
        service_rates=service_rates,
        thetas=thetas,
        bound=bound,
    )
    probs = models.get_state_probabilities(
        num_classes=num_classes,
        num_servers=num_servers,
        arrival_rates=arrival_rates,
        service_rates=service_rates,
        thetas=thetas,
        bound=bound,
    )
    v = models.find_var_sojourn_time(
        State_Space=state_space,
        transition_matrix=transmat,
        arrival_rates=arrival_rates,
        probs=probs,
    )
    return v

In [25]:
def get_sojourn_var_sim(
    num_classes, num_servers, arrival_rates, service_rates, thetas, max_simulation_time
):
    Q = models.build_and_run_simulation(
        num_classes=num_classes,
        num_servers=num_servers,
        arrival_rates=arrival_rates,
        service_rates=service_rates,
        class_change_rate_matrix=thetas,
        max_simulation_time=max_simulation_time,
        progress_bar=True,
    )
    recs = Q.get_all_records()
    sojourn_times = [
        r.exit_date - r.arrival_date
        for r in recs
        if r.arrival_date > max_simulation_time * 0.2
        if r.record_type == "service"
    ]
    return np.var(sojourn_times)

In [26]:
def compare_sojourn_var(
    num_classes,
    num_servers,
    arrival_rates,
    service_rates,
    thetas,
    max_simulation_time,
    bound,
):
    v_sim = get_sojourn_var_sim(
        num_classes,
        num_servers,
        arrival_rates,
        service_rates,
        thetas,
        max_simulation_time,
    )
    v_mc = get_sojourn_var_mc(
        num_classes, num_servers, arrival_rates, service_rates, thetas, bound
    )
    return {"Markov Chain": v_mc, "Simulation": v_sim}

In [28]:
compare_sojourn_var(
    num_classes=2,
    num_servers=2,
    arrival_rates=[1.5, 1.5],
    service_rates=[2.5, 3.5],
    thetas=[[None, 1.0], [2.0, None]],
    max_simulation_time=3000,
    bound=10,
)

100%|█████████████████████████████████| 3000.0/3000 [00:00<00:00, 3823.91it/s]


{'Markov Chain': 0.3495886745739521, 'Simulation': 0.3498846437419317}

In [37]:
compare_sojourn_var(
    num_classes=2,
    num_servers=2,
    arrival_rates=[1.0, 2.5],
    service_rates=[3.0, 3.5],
    thetas=[[None, 2.0], [2.5, None]],
    max_simulation_time=10000,
    bound=12,
)

100%|███████████████████████████████| 10000.0/10000 [00:04<00:00, 2093.07it/s]


{'Markov Chain': 0.24900063102604736, 'Simulation': 0.2404044398737076}

In [38]:
compare_sojourn_var(
    num_classes=3,
    num_servers=2,
    arrival_rates=[1.0, 2.5, 1.0],
    service_rates=[4.0, 3.5, 4.5],
    thetas=[[None, 2.0, 1.0], [2.5, None, 1.0], [0.5, 0.5, None]],
    max_simulation_time=10000,
    bound=8,
)

100%|███████████████████████████████| 10000.0/10000 [00:04<00:00, 2486.99it/s]


{'Markov Chain': 0.20418204962796385, 'Simulation': 0.20685988903162209}